In [1]:
import pandas as pd
import numpy as np
import re
import time
import random
import gensim
import jieba
from numpy import *
import json
from numpy import linalg as la  

E:\Applications\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [57]:
info=pd.read_csv('info2.csv',encoding='gbk')

In [3]:
newdian=pd.read_csv('点播信息2.csv',encoding='gbk')

In [4]:
newtv=pd.read_csv('附件2：电视产品信息数据（新出现）.csv',encoding='gbk')

In [62]:
newlist=[]

In [63]:
for i in range(len(newdian)):
    st=newdian.loc[i,'影片名称']
    st=re.sub('：.*','',st)
    st=re.sub('\(.+\)','',st)
    st=re.sub('【.+】','',st)
    st=re.sub('（.+）','',st)
    st=re.sub('\S+月\S+日','',st)
    st=re.sub('第.季','',st)
    st=re.sub('Ⅱ','',st)
    st=re.sub('Ⅲ','',st)
    st=re.sub('\d{8}','',st)
    st=re.sub('\?.*\?','',st)
    st=re.sub('\?','',st)
    st=re.sub('？','',st)
    st=re.sub('·','',st)
    #st=re.sub(' ','',st)
    st=st.strip()
    newlist.append(st)

In [64]:
for i in range(len(newtv)):
    st=newtv.loc[i,'正题名']
    st=re.sub('：.*','',st)
    st=re.sub('\(.+\)','',st)
    st=re.sub('【.+】','',st)
    st=re.sub('（.+）','',st)
    st=re.sub('\S+月\S+日','',st)
    st=re.sub('第.+季','',st)
    st=re.sub('Ⅱ','',st)
    st=re.sub('Ⅲ','',st)
    st=re.sub('\d{8}','',st)
    st=re.sub('\?.*\?','',st)
    st=re.sub('\?','',st)
    st=re.sub('？','',st)
    st=re.sub('·','',st)
    #st=re.sub(' ','',st)
    st=st.strip()
    newlist.append(st)

In [65]:
newlist=list(set(newlist))

In [66]:
newlist2=[]
for n in newlist:
    if not re.search('广告',n):
        newlist2.append(n)

In [67]:
newlist2

['',
 '音悦V榜港台',
 '极速风流',
 '贝肯熊',
 '蓝海少女',
 'CP52-游子吟',
 'CP81-皮影戏',
 'CP46-愧',
 '黑白照相馆',
 '星际小蚂蚁大学堂之五常文化系列机智',
 '星际小蚂蚁大学堂之五常文化系列侠义',
 '撒娇女人最好命',
 '等风来',
 '功夫熊猫3',
 '积高侠',
 '每日妈妈',
 '剪辑',
 'CP01-以爱之名',
 '钱多多嫁人记',
 '乘胜狙击',
 'CP84-野孩子',
 'CP40-Come Along',
 '樱桃小丸子',
 'CP08-Memories',
 '万万没想到',
 '英雄本色1',
 '猪猪侠光明守卫者',
 '2017亚洲羽毛球锦标赛-决赛',
 '冬日奇缘',
 'CP18-与我常在',
 '星际小蚂蚁3D儿歌系列我爱北京天安门',
 'CP84-美丽到不能',
 'CP47-Please Dont Say You Love Me',
 '甜果屋宣传汇总161129',
 'CP21-Touch',
 '贝瓦儿歌精选集洋娃娃和小熊跳舞',
 '央视新闻联播',
 '小济公',
 '小老虎丹尼尔 下',
 '星际小蚂蚁大学堂之五常文化系列守义',
 'CP96-士多啤梨苹果橙',
 'CP72-Acapella',
 'CP76-小女孩',
 '魔晶猎人',
 'CP09-我的骄傲',
 '2017年北京卫视跨年环球歌会',
 'CP60-直角等于三角',
 '猛鬼街',
 '中华小当家',
 'CP94-伤信',
 '托马斯和朋友',
 '爱，来的刚好',
 '我的媳妇是女王',
 '熊仔',
 'CP42-正气歌',
 '72家房客',
 'CP00-失忆周末',
 '星际小蚂蚁之环球追梦被欺骗的友谊',
 '绿巨人2',
 '购物狂想曲',
 'CP49-让世界为你转身',
 '星际小蚂蚁公益大使系列动画电梯常识4',
 '海珠区综治平安创建巡礼',
 '飞哥大英雄',
 'CP20-迷你',
 '红衣小超人科罗拉多',
 '辛普森一家',
 '熊出没',
 'CP03-Figure 8',
 '合约男女',
 '养生堂',
 '猪猪侠之超星萌宠三',
 '碧海追踪',
 '161121-1甜果电视升级篇',
 '古战场传奇',
 '危思禁入'

In [68]:
len(newlist2)

3346

In [69]:
for i in range(len(newlist2)):
    info.loc[i+len(info),'name']=newlist2[i]

In [70]:
rawmlist=info.drop_duplicates(['name']).reset_index(drop=True)

In [72]:
rawmlist=rawmlist.fillna('')

In [112]:
for i in range(len(rawmlist)):
    if rawmlist.loc[i,'info']=='':
        if re.search('新闻',rawmlist.loc[i,'name']) or re.search('视角',rawmlist.loc[i,'name']) or re.search('榜样',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['新闻'])
        if re.search('儿歌',rawmlist.loc[i,'name']) or re.search('宝宝',rawmlist.loc[i,'name']) or re.search('赛尔',rawmlist.loc[i,'name']) or re.search('如意兔',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童','动画'])
        if re.search('CP',rawmlist.loc[i,'name']) or re.search('演唱会',rawmlist.loc[i,'name']) or re.search('音',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['音乐'])
        if re.search('联赛',rawmlist.loc[i,'name']) or re.search('大奖赛',rawmlist.loc[i,'name']) or re.search('锦标赛',rawmlist.loc[i,'name']) or re.search('世锦赛',rawmlist.loc[i,'name']) or re.search('赛季',rawmlist.loc[i,'name']) or re.search('运动会',rawmlist.loc[i,'name']) or re.search('公开赛',rawmlist.loc[i,'name']) or re.search('全运会',rawmlist.loc[i,'name']) or re.search('亚冠',rawmlist.loc[i,'name']) or re.search('NBA',rawmlist.loc[i,'name']) or re.search('CBA',rawmlist.loc[i,'name'])  or re.search('足',rawmlist.loc[i,'name']) or re.search('球赛',rawmlist.loc[i,'name']) or re.search('冠军杯',rawmlist.loc[i,'name']) or re.search('乒乓',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['体育'])
        if re.search('少儿',rawmlist.loc[i,'name']) or re.search('选举',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童','家庭','真人秀'])
        if re.search('健康',rawmlist.loc[i,'name']) or re.search('养生',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['真人秀'])
        if re.search('开心球',rawmlist.loc[i,'name']) or re.search('如意酷宝',rawmlist.loc[i,'name']) or re.search('大头儿子',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童','家庭','动画'])
        if re.search('才艺',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['真人秀'])
        if re.search('甜果',rawmlist.loc[i,'name']) or re.search('大仓库',rawmlist.loc[i,'name']) or re.search('快乐大巴',rawmlist.loc[i,'name']) or re.search('芝麻街',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童','家庭'])
        if re.search('秀',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['脱口秀'])
        if re.search('方大同',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['音乐','真人秀'])
        if re.search('大业',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['剧情','动作','历史'])
        if re.search('自然',rawmlist.loc[i,'name']) or re.search('地球',rawmlist.loc[i,'name']) or re.search('探索',rawmlist.loc[i,'name']) or re.search('特别呈现',rawmlist.loc[i,'name']) or re.search('建筑',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['纪录片'])
        if re.search('年级',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童'])
        if re.search('档案',rawmlist.loc[i,'name']) or re.search('解密',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['纪录片','历史'])
        if re.search('秦宫冷月',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['戏曲'])
        if re.search('我是未来',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['科幻','真人秀','脱口秀'])
        if re.search('星际小蚂蚁',rawmlist.loc[i,'name']) or re.search('超级玛丽',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['儿童','动画','冒险'])
        if re.search('恶魔',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['惊悚'])
        if re.search('加菲猫',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['喜剧','动画'])
        if re.search('网球王子',rawmlist.loc[i,'name']):
            rawmlist.loc[i,'info']=json.dumps(['动作','喜剧','运动'])

In [114]:
for i in range(len(rawmlist)):
    if rawmlist.loc[i,'info']=='':
        rawmlist.loc[i,'info']=json.dumps('[]')

In [115]:
rawmlist.loc[rawmlist.loc[:,'info']=='']

,连续剧分类,出品年代,info,rname,name,url,year,subtype,rate,id,img


In [84]:
tvlist=rawmlist.loc[rawmlist.loc[:,'info']==''].loc[:,'name'].tolist()[1:]

In [85]:
tvlist

['剪辑',
 '猪猪侠光明守卫者',
 '魔晶猎人',
 '2017年北京卫视跨年环球歌会',
 '海珠区综治平安创建巡礼',
 '红衣小超人科罗拉多',
 '辛普森一家',
 '梦想合伙人',
 '加菲猫和他的朋友们 肮脏的生意',
 'SUNDAY',
 '潮语动画片',
 '170811-02《鲛珠传》宣传片',
 '猪猪侠之梦想守卫者 下部',
 '红色护卫北京高清',
 '让世界听见',
 '170703-01河北卫视《家政女皇》',
 '阿U神奇萝卜三',
 'AQ少年之使命召唤',
 '如意兔之开心农场Ⅴ失忆药水',
 '新大头儿子和小头爸爸之超能爸爸',
 '大唐双龙传',
 '如意兔之开心农场大速生菜',
 '花样爷爷',
 '加菲猫和他的朋友们 罗伊的生日',
 '如意兔之开心农场Ⅴ影子武士',
 '大片',
 '加菲猫和他的朋友们 加菲心爱的床',
 '加菲猫和他的朋友们 拉萨格纳伯爵',
 '醉侠苏乞儿',
 '如意兔之开心农场跳来跳去的小鱼',
 '路易斯泰鲁',
 '超级玛丽的世界思库帕库帕大王',
 '南京！南京！',
 '如意兔之开心农场长翅膀的树',
 '第一滴血',
 '新网球王子TV版王子归来',
 '加菲猫和他的朋友们 欧迪和三只猫的故事',
 '如意兔之开心农场木瓜的小蜜蜂',
 '风暴里的摇篮',
 '森美旅行团',
 '如意兔之开心农场强盗蟹',
 '最美街景百盆鲜花进驻仰忠',
 '钢铁侠系列1和2',
 '170118-02学科辅导高清',
 '如意兔之开心农场大停电',
 '小马宝莉特别篇 迷人的生日',
 '新大头儿子和小头爸爸2',
 '芬芳簇拥世界',
 '如意兔之开心农场Ⅴ瓢虫监听器',
 '太乙仙魔录之灵飞纪+',
 '亲爱',
 '如意兔之开心农场最危险的树',
 '170504-1东方卫视《欢乐颂2》宣传片',
 '灵魂战车',
 '加菲猫和他的朋友们 哞哞叫的小狗',
 '如意兔之开心农场Ⅴ地雷战',
 '如意兔之开心农场Ⅴ蟾蜍大聚会',
 '阿U兔智来了',
 '后备箱',
 '智力快车',
 '超级玛丽的世界天生骑手',
 '《鲁滨逊漂流记》片花',
 '170525-02《基层治 南粤安》',
 '行走的筷子宁夏篇',
 '如意兔之开心农场顽皮的小松鼠',
 '综艺精彩50年我爱EYT',


In [86]:
import re
import requests, time
from bs4 import BeautifulSoup

fujian2_set = tvlist
dic={}

url = 'https://www.baidu.com/s?ie=utf-8&f=8&rsv_bp=1&tn=baidu&wd={}'

headers = {
    'Connection': 'close',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}


for jiemu in fujian2_set:
    print(jiemu)
    flag = 0
    res = requests.get(url.format(jiemu), headers=headers)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    nowl=[]
    try:
        for s in soup.find_all('p', {'class':'op-zx-new-mvideo-oneline'}):
            if s.find('span').text == '类型：':
                dic[jiemu]=[temp.text for temp in s.find_all('a')]
                flag = 1
                break
    except:
        pass

    try:
        if flag == 0:
            for s in soup.find_all('p', {'class':'op-zx-new-tvideo-oneline'}):
                if s.find('span').text == '类型：':
                    dic[jiemu]=[temp.text for temp in s.find_all('a')]
                    flag = 1
                    break
    except:
        pass
    try:
        print(dic[jiemu])
    except Exception as e:
        print(e)


剪辑
'剪辑'
猪猪侠光明守卫者
'猪猪侠光明守卫者'
魔晶猎人
['魔幻']
2017年北京卫视跨年环球歌会
'2017年北京卫视跨年环球歌会'
海珠区综治平安创建巡礼
'海珠区综治平安创建巡礼'
红衣小超人科罗拉多
'红衣小超人科罗拉多'
辛普森一家
'辛普森一家'
梦想合伙人
['都市', '商战', '青春']
加菲猫和他的朋友们 肮脏的生意
'加菲猫和他的朋友们 肮脏的生意'
SUNDAY
'SUNDAY'
潮语动画片
'潮语动画片'
170811-02《鲛珠传》宣传片
'170811-02《鲛珠传》宣传片'
猪猪侠之梦想守卫者 下部
['童话']
红色护卫北京高清
'红色护卫北京高清'
让世界听见
'让世界听见'
170703-01河北卫视《家政女皇》
'170703-01河北卫视《家政女皇》'
阿U神奇萝卜三
'阿U神奇萝卜三'
AQ少年之使命召唤
'AQ少年之使命召唤'
如意兔之开心农场Ⅴ失忆药水
'如意兔之开心农场Ⅴ失忆药水'
新大头儿子和小头爸爸之超能爸爸
'新大头儿子和小头爸爸之超能爸爸'
大唐双龙传
'大唐双龙传'
如意兔之开心农场大速生菜
'如意兔之开心农场大速生菜'
花样爷爷
'花样爷爷'
加菲猫和他的朋友们 罗伊的生日
'加菲猫和他的朋友们 罗伊的生日'
如意兔之开心农场Ⅴ影子武士
'如意兔之开心农场Ⅴ影子武士'
大片
'大片'
加菲猫和他的朋友们 加菲心爱的床
'加菲猫和他的朋友们 加菲心爱的床'
加菲猫和他的朋友们 拉萨格纳伯爵
'加菲猫和他的朋友们 拉萨格纳伯爵'
醉侠苏乞儿
'醉侠苏乞儿'
如意兔之开心农场跳来跳去的小鱼
'如意兔之开心农场跳来跳去的小鱼'
路易斯泰鲁
'路易斯泰鲁'
超级玛丽的世界思库帕库帕大王
'超级玛丽的世界思库帕库帕大王'
南京！南京！
'南京！南京！'
如意兔之开心农场长翅膀的树
'如意兔之开心农场长翅膀的树'
第一滴血
'第一滴血'
新网球王子TV版王子归来
'新网球王子TV版王子归来'
加菲猫和他的朋友们 欧迪和三只猫的故事
'加菲猫和他的朋友们 欧迪和三只猫的故事'
如意兔之开心农场木瓜的小蜜蜂
'如意兔之开心农场木瓜的小蜜蜂'
风暴里的摇篮
'风暴里的摇篮'
森美旅行团
'森美旅行团'
如意兔之开心农场强盗蟹
'如意兔之开心农场强盗蟹'
最美街景百盆鲜花进驻仰忠
'最美街景百盆鲜花进驻仰忠'
钢

'爱情心心相印'
厝边头尾
'厝边头尾'
ReLife
'ReLife'
美丽乡村
'美丽乡村'
幸腹涂鸦
['搞笑', '日常', '美食']
嘟拉安全教育
'嘟拉安全教育'
冲出网围-广州市第七届电教片三等奖
'冲出网围-广州市第七届电教片三等奖'
170118-4学科辅导标清
'170118-4学科辅导标清'
天河先进党员事迹汇报短片
'天河先进党员事迹汇报短片'
如意兔之开心农场Ⅴ超级武器
'如意兔之开心农场Ⅴ超级武器'
如意兔之开心农场Ⅴ毒气弹
'如意兔之开心农场Ⅴ毒气弹'
如意兔之开心农场神秘的大眼睛
'如意兔之开心农场神秘的大眼睛'
第一次
'第一次'
伟大的苏格兰人
'伟大的苏格兰人'
如意兔之开心农场Ⅴ竹子要开花
'如意兔之开心农场Ⅴ竹子要开花'
好好先生
'好好先生'
赛车总动员3
'赛车总动员3'
声之形
'声之形'
平安海珠高清版
'平安海珠高清版'
AR神秘画册
'AR神秘画册'
火星研究院
'火星研究院'
为国王和帝国而战
'为国王和帝国而战'
醉拳
'醉拳'
如意兔之开心农场Ⅴ焰火树
'如意兔之开心农场Ⅴ焰火树'
比利林恩的中场战事
['战争']
精灵鼠小弟
'精灵鼠小弟'
热诚传说X
['热血', '神魔']
高海拔之恋2
'高海拔之恋2'
如意兔之开心农场Ⅴ强力火药筒
'如意兔之开心农场Ⅴ强力火药筒'
蚂蚁公主2
'蚂蚁公主2'
如意兔之开心农场Ⅴ危险的萝卜
'如意兔之开心农场Ⅴ危险的萝卜'
加菲猫和他的朋友们 木星仙人掌传奇
'加菲猫和他的朋友们 木星仙人掌传奇'
宝狄与好友梦想联盟
'宝狄与好友梦想联盟'
看我72变
'看我72变'
如意兔之开心农场致命仙人掌
'如意兔之开心农场致命仙人掌'
170627-03升级鞋
'170627-03升级鞋'
加菲猫和他的朋友们 都市牛仔
'加菲猫和他的朋友们 都市牛仔'
剩女也疯狂
'剩女也疯狂'
球手们-片花
'球手们-片花'
如意兔之开心农场Ⅴ玉米塔
'如意兔之开心农场Ⅴ玉米塔'
如意兔之开心农场意外发明
'如意兔之开心农场意外发明'
浮沉
'浮沉'
爱回家之开心速递
'爱回家之开心速递'
爱情自有天意
'爱情自有天意'
最强战士迷你特工之英雄的诞生
'最强战士迷你特工之英雄的诞生'
最美街景广州建绿道3200公里
'最美街景广州建绿道3200公里'
《冲天火》片花
'《冲天火》片花'

In [53]:
typelist=['犯罪', '喜剧', '舞台艺术', '新闻', '儿童', '真人秀', '歌舞', '戏曲', '音乐', '冒险', '灾难', '恐怖', '家庭', '短片', '历史', '运动', '动画', '纪录片', '惊悚', '科幻', '战争', '动作', '健康', '脱口秀', '悬疑', '剧情', '古装', '鬼怪', '奇幻', '武侠', '同性', '体育', '西部', '传记', '爱情']
typelist=sorted(typelist)

In [110]:
dic=dic2
for i in range(2660,len(rawmlist)):
    nowtag=[]
    if rawmlist.loc[i,'name'] in dic.keys():
        for j in dic[rawmlist.loc[i,'name']]:
            if j in typelist:
                nowtag.append(j)
            elif j=='亲子' or j=='益智':
                nowtag+=['儿童','家庭']
            elif j=='宫廷':
                nowtag.append('古装')
            elif j=='悬疑剧':
                nowtag.append('悬疑')
            elif j=='言情' or j=='情感':
                nowtag.append('爱情')
            elif j=='搞笑':
                nowtag.append('喜剧')
            elif j=='童话':
                nowtag.append('儿童')
            elif j=='魔幻':
                nowtag.append('奇幻')
    nowtag=list(set(nowtag))
    print(i,nowtag)
    if len(nowtag)>0:
        rawmlist.loc[i,'info']=json.dumps(nowtag)

2660 []
2661 []
2662 []
2663 []
2664 []
2665 []
2666 []
2667 []
2668 []
2669 []
2670 []
2671 []
2672 []
2673 []
2674 []
2675 []
2676 []
2677 []
2678 []
2679 []
2680 []
2681 []
2682 []
2683 []
2684 []
2685 []
2686 []
2687 []
2688 ['奇幻', '冒险', '喜剧']
2689 []
2690 []
2691 []
2692 []
2693 []
2694 []
2695 []
2696 []
2697 []
2698 []
2699 []
2700 []
2701 []
2702 []
2703 []
2704 []
2705 []
2706 []
2707 []
2708 []
2709 []
2710 []
2711 []
2712 []
2713 []
2714 []
2715 []
2716 []
2717 []
2718 []
2719 []
2720 ['剧情', '爱情']
2721 []
2722 []
2723 []
2724 []
2725 []
2726 []
2727 []
2728 []
2729 []
2730 []
2731 []
2732 []
2733 []
2734 []
2735 []
2736 []
2737 []
2738 []
2739 []
2740 []
2741 []
2742 []
2743 []
2744 []
2745 []
2746 []
2747 []
2748 []
2749 []
2750 []
2751 []
2752 []
2753 []
2754 []
2755 []
2756 []
2757 []
2758 []
2759 []
2760 []
2761 []
2762 []
2763 []
2764 []
2765 []
2766 []
2767 []
2768 []
2769 []
2770 []
2771 []
2772 []
2773 []
2774 []
2775 ['奇幻']
2776 []
2777 []
2778 []
2779 []
2780 ['动画'

4362 []
4363 []
4364 []
4365 ['剧情']
4366 []
4367 []
4368 []
4369 []
4370 []
4371 []
4372 []
4373 []
4374 []
4375 []
4376 []
4377 ['奇幻', '冒险', '家庭', '喜剧']
4378 []
4379 []
4380 []
4381 []
4382 []
4383 []
4384 []
4385 ['喜剧']
4386 []
4387 []
4388 []
4389 []
4390 []
4391 []
4392 []
4393 []
4394 []
4395 []
4396 []
4397 []
4398 []
4399 []
4400 []
4401 []
4402 []
4403 []
4404 []
4405 []
4406 []
4407 []
4408 []
4409 []
4410 []
4411 []
4412 []
4413 []
4414 []
4415 []
4416 []
4417 []
4418 []
4419 []
4420 []
4421 []
4422 []
4423 []
4424 []
4425 []
4426 []
4427 []
4428 []
4429 []
4430 []
4431 ['爱情']
4432 []
4433 []
4434 []
4435 []
4436 []
4437 []
4438 []
4439 []
4440 []
4441 []
4442 []
4443 []
4444 []
4445 []
4446 []
4447 []
4448 []
4449 []
4450 []
4451 []
4452 []
4453 []
4454 []
4455 []
4456 []
4457 []
4458 []
4459 []
4460 []
4461 []
4462 []
4463 []
4464 []
4465 []
4466 []
4467 []
4468 []
4469 []
4470 []
4471 []
4472 []
4473 []
4474 []
4475 []
4476 []
4477 []
4478 []
4479 []
4480 []
4481 []
4482 [

In [78]:
rawmlist

,连续剧分类,出品年代,info,rname,name,url,year,subtype,rate,id,img
0,职场是个技术活,2017,"[""剧情"", ""爱情""]",职场是个技术活,职场是个技术活,https://movie.douban.com/subject/26108041/,2017,tv,"{""stars"": ""25"", ""min"": 0, ""max"": 10, ""average""...",2.6108e+07,http://img7.doubanio.com/view/photo/s_ratio_po...
1,自然,2017,"[""剧情"", ""奇幻"", ""恐怖""]",邪恶力量 第十三季,自然,https://movie.douban.com/subject/26952093/,2017,tv,"{""stars"": ""45"", ""min"": 0, ""max"": 10, ""average""...",2.69521e+07,http://img7.doubanio.com/view/photo/s_ratio_po...
2,深海利剑,2017,"[""剧情""]",深海利剑,深海利剑,https://movie.douban.com/subject/26879740/,2017,tv,"{""stars"": ""30"", ""min"": 0, ""max"": 10, ""average""...",2.68797e+07,http://img7.doubanio.com/view/photo/s_ratio_po...
3,金星秀(热),2017,"[""脱口秀""]",金星秀,金星秀,https://movie.douban.com/subject/26314504/,2015,tv,"{""stars"": ""40"", ""min"": 0, ""max"": 10, ""average""...",2.63145e+07,http://img3.doubanio.com/view/photo/s_ratio_po...
4,(热)赛小花的远大前程,2017,"[""剧情"", ""爱情""]",赛小花的远大前程,赛小花的远大前程,https://movie.douban.com/subject/26772028/,2017,tv,"{""stars"": ""30"", ""min"": 0, ""max"": 10, ""average""...",2.6772e+07,http://img7.doubanio.com/view/photo/s_ratio_po...
5,(热)人民的名义,2017,"[""剧情"", ""犯罪"", ""悬疑""]",人民的名义,人民的名义,https://movie.douban.com/subject/26727273/,2017,tv,"{""stars"": ""45"", ""min"": 0, ""max"": 10, ""average""...",2.67273e+07,http://img3.doubanio.com/view/photo/s_ratio_po...
6,(热)择天记,2017,"[""剧情"", ""奇幻"", ""古装""]",择天记,择天记,https://movie.douban.com/subject/26614082/,2017,tv,"{""stars"": ""20"", ""min"": 0, ""max"": 10, ""average""...",2.66141e+07,http://img3.doubanio.com/view/photo/s_ratio_po...
7,心理追凶,2016,"[""剧情"", ""科幻"", ""犯罪""]",信号,心理追凶,https://movie.douban.com/subject/26310143/,2016,tv,"{""stars"": ""45"", ""min"": 0, ""max"": 10, ""average""...",2.63101e+07,http://img3.doubanio.com/view/photo/s_ratio_po...
8,全职没女,2016,"[""剧情"", ""喜剧"", ""爱情""]",全职没女,全职没女,https://movie.douban.com/subject/26726989/,2016,tv,"{""stars"": ""40"", ""min"": 0, ""max"": 10, ""average""...",2.6727e+07,http://img3.doubanio.com/view/photo/s_ratio_po...
9,糊涂县令郑板桥,2016,"[""剧情"", ""古装""]",糊涂县令郑板桥,糊涂县令郑板桥,https://movie.douban.com/subject/26365498/,2016,tv,"{""stars"": ""30"", ""min"": 0, ""max"": 10, ""average""...",2.63655e+07,http://img7.doubanio.com/view/photo/s_ratio_po...


In [108]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36"}
clist=['中文名','外文名','其它译名','出品公司', '制片地区', '拍摄地点', '发行公司', '首播时间', '导演', '编剧', '主演', '集数',
       '每集长度', '类型', '上映时间', '在线播放平台', '首播平台', '主题曲', '播出时间']
dic2={}
from bs4 import BeautifulSoup as BS
for i in range(len(tvlist)):#len(tvlist)
    print(i)
    url="https://baike.baidu.com/item/"
    print(url+tvlist[i].split()[0])
    try:
        res=requests.get(url+tvlist[i],headers=headers,timeout=5)
        res.encoding='utf-8'
    except Exception as e:
        print(e)
        continue
    soup=BS(res.text,'lxml')
    if soup.title.text=="百度百科——全球最大中文百科全书":
        continue
    else:
        if soup.find('div',class_='basic-info'):
            dlist=soup.find('div',class_='basic-info').find_all('dt')
            for d in dlist:
                if d.text.replace('\xa0','') =='类型':
                    dic2[tvlist[i]]=re.split('(、|,|，|；|/|\s)',re.sub('\xa0','',d.next_sibling.next_sibling.text.strip()))
                    break

0
https://baike.baidu.com/item/剪辑
1
https://baike.baidu.com/item/猪猪侠光明守卫者
2
https://baike.baidu.com/item/魔晶猎人
3
https://baike.baidu.com/item/2017年北京卫视跨年环球歌会
4
https://baike.baidu.com/item/海珠区综治平安创建巡礼
5
https://baike.baidu.com/item/红衣小超人科罗拉多
6
https://baike.baidu.com/item/辛普森一家
7
https://baike.baidu.com/item/梦想合伙人
8
https://baike.baidu.com/item/加菲猫和他的朋友们
9
https://baike.baidu.com/item/SUNDAY
10
https://baike.baidu.com/item/潮语动画片
11
https://baike.baidu.com/item/170811-02《鲛珠传》宣传片
12
https://baike.baidu.com/item/猪猪侠之梦想守卫者
13
https://baike.baidu.com/item/红色护卫北京高清
14
https://baike.baidu.com/item/让世界听见
15
https://baike.baidu.com/item/170703-01河北卫视《家政女皇》
16
https://baike.baidu.com/item/阿U神奇萝卜三
17
https://baike.baidu.com/item/AQ少年之使命召唤
18
https://baike.baidu.com/item/如意兔之开心农场Ⅴ失忆药水
19
https://baike.baidu.com/item/新大头儿子和小头爸爸之超能爸爸
20
https://baike.baidu.com/item/大唐双龙传
21
https://baike.baidu.com/item/如意兔之开心农场大速生菜
22
https://baike.baidu.com/item/花样爷爷
23
https://baike.baidu.com/item/加菲猫和他的朋友们
24
http

193
https://baike.baidu.com/item/继承人大结局
194
https://baike.baidu.com/item/最美街景广州二沙岛重磅变身
195
https://baike.baidu.com/item/如意兔之开心农场生活啊生活
196
https://baike.baidu.com/item/历史偶像
197
https://baike.baidu.com/item/辛亥革命
198
https://baike.baidu.com/item/最美街景白云寮采村
199
https://baike.baidu.com/item/师奶永远OK
200
https://baike.baidu.com/item/快乐星球Ⅳ
201
https://baike.baidu.com/item/憨豆先生的黄金周(异样逋纠追
202
https://baike.baidu.com/item/如意兔之开心农场Ⅴ一棵酒瓶树引发的灾难
203
https://baike.baidu.com/item/珠江
204
https://baike.baidu.com/item/索玛花开
205
https://baike.baidu.com/item/如意兔之开心农场Ⅴ龙卷风
206
https://baike.baidu.com/item/故宫
207
https://baike.baidu.com/item/暴力街区赛车
208
https://baike.baidu.com/item/穿普拉达的恶魔
209
https://baike.baidu.com/item/红色娘子军
210
https://baike.baidu.com/item/憨豆特工
211
https://baike.baidu.com/item/最美街景06
212
https://baike.baidu.com/item/新大头儿子和小头爸爸之梦幻乐园奇遇记
213
https://baike.baidu.com/item/雀圣3自摸三百番
214
https://baike.baidu.com/item/如意兔之开心农场牙刷树
215
https://baike.baidu.com/item/冲浪企鹅2
216
https://baike.baidu.com/item/英语

387
https://baike.baidu.com/item/加菲猫和他的朋友们
388
https://baike.baidu.com/item/剩女也疯狂
389
https://baike.baidu.com/item/球手们-片花
390
https://baike.baidu.com/item/如意兔之开心农场Ⅴ玉米塔
391
https://baike.baidu.com/item/如意兔之开心农场意外发明
392
https://baike.baidu.com/item/浮沉
393
https://baike.baidu.com/item/爱回家之开心速递
394
https://baike.baidu.com/item/爱情自有天意
395
https://baike.baidu.com/item/最强战士迷你特工之英雄的诞生
396
https://baike.baidu.com/item/最美街景广州建绿道3200公里
397
https://baike.baidu.com/item/《冲天火》片花
398
https://baike.baidu.com/item/再现历史
399
https://baike.baidu.com/item/夸世代
400
https://baike.baidu.com/item/明星爱厨房
401
https://baike.baidu.com/item/如意兔之开心农场低碳生活
402
https://baike.baidu.com/item/齐癫大圣福禄寿
403
https://baike.baidu.com/item/如意兔之开心农场Ⅴ魔豆
404
https://baike.baidu.com/item/爱回家之开心速递
405
https://baike.baidu.com/item/辽宁卫视春节联欢晚会2016
406
https://baike.baidu.com/item/圆桌派
407
https://baike.baidu.com/item/哗鬼家族
408
https://baike.baidu.com/item/咱们穿越吧
409
https://baike.baidu.com/item/永无止境
410
https://baike.baidu.com/item/来电不善
411


In [116]:
rawmlist.to_csv('info3.csv',index=False)